# **Video Processing Week 2: Analisis Manusia (Wajah & Tubuh) dengan MediaPipe**

Minggu ini kita akan mempelajari **analisis manusia dalam video** menggunakan teknologi AI melalui MediaPipe. Fokus utama adalah deteksi dan tracking fitur wajah serta pose tubuh manusia secara real-time dengan akurasi tinggi.

**Tujuan Pembelajaran:**

Di akhir sesi ini kita akan mampu:
- Menggunakan model AI (via MediaPipe) untuk mengekstrak fitur wajah dan tubuh manusia secara real-time
- Mendeteksi wajah dan 468 titik landmark pada wajah dengan presisi tinggi
- Mengimplementasikan aplikasi deteksi kedipan mata menggunakan Eye Aspect Ratio (EAR)
- Mendeteksi 33 titik landmark pada tubuh manusia (skeleton/pose detection)
- Membangun aplikasi computer vision praktis untuk analisis gerakan manusia

**Topik Praktik:**
- **Pengenalan MediaPipe**: Setup dan penggunaan library MediaPipe untuk solusi AI real-time
- **Facial Landmark Detection**: Deteksi 468 titik wajah dan aplikasi deteksi kedipan mata
- **Pose Landmark Detection**: Deteksi skeleton tubuh dan analisis sudut sendi untuk deteksi gerakan

> *This module is inspired by the development of last semester’s materials.* 

## **Pengenalan MediaPipe**

MediaPipe adalah framework open-source dari Google yang dirancang untuk membangun pipeline pemrosesan media secara real-time, seperti visi komputer dan pengolahan audio. MediaPipe menyediakan model deteksi wajah yang ringan, akurat, dan cepat, yang dapat digunakan baik untuk gambar statis maupun video streaming real-time.

*Sebelum lanjut, kita coba import library yang akan kita butuhkan dulu*

**⚠️ PENTING: Jika anda menggunakan library opencv-contrib-python dari materi minggu lalu, pastikan menggunakan versi 4.11.0.86 untuk menghindari masalah kompatibilitas dengan Mediapipe.**

In [1]:
# pip install opencv-python numpy matplotlib mediapipe ipykernel
# atau
%pip install opencv-contrib-python==4.11.0.86 numpy matplotlib mediapipe ipykernel

Note: you may need to restart the kernel to use updated packages.


c:\Users\muham\OneDrive\Desktop\sistem-teknologi-multimedia-122140193\multimedia-uv\Scripts\python.exe: No module named pip


In [2]:
import cv2
import numpy as np
import mediapipe as mp
import matplotlib.pyplot as plt
import os

**Penjelasan Fungsi Penting:**
* **cv2** (OpenCV): Library utama untuk semua operasi CV: membaca/menulis video, konversi warna, filter, dan object tracking.
* **numpy**: Pondasi untuk komputasi numerik, digunakan untuk memanipulasi frame video (yang merupakan array).
* **matplotlib**: Digunakan untuk menampilkan gambar atau frame video di dalam output cell Python Notebook.

### Menggunakan Mediapipe Real-time Face Mesh Detection

In [3]:
PATH_VIDEO = os.path.join(os.getcwd(), 'data', 'man_walking.mp4')

# Inisialisasi MediaPipe Face Detection
mp_face = mp.solutions.face_detection
mp_draw = mp.solutions.drawing_utils

# Model selection 0 untuk jarak dekat, 1 untuk jarak jauh
face_det = mp_face.FaceDetection(model_selection=1, min_detection_confidence=0.5)

# Buka video
cap = cv2.VideoCapture(PATH_VIDEO)
if not cap.isOpened():
    raise RuntimeError(f"Gagal membuka video: {PATH_VIDEO}")

while True:
    ok, frame = cap.read()
    if not ok:
        break

    height, width = frame.shape[:2]

    # MediaPipe butuh RGB
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = face_det.process(rgb)

    # Gambar bounding box
    if result.detections:
        for det in result.detections:
            rel_box = det.location_data.relative_bounding_box
            x = int(rel_box.xmin * width)
            y = int(rel_box.ymin * height)
            bw = int(rel_box.width * width)
            bh = int(rel_box.height * height)

            # Clamp koordinat
            x = max(0, x); y = max(0, y)
            bw = max(0, min(bw, width - x))
            bh = max(0, min(bh, height - y))

            cv2.rectangle(frame, (x, y), (x + bw, y + bh), (0, 255, 0), 2)

            # (Opsional) tampilkan score
            if det.score:
                score_txt = f"{det.score[0]:.2f}"
                cv2.putText(frame, score_txt, (x, max(0, y - 8)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2, cv2.LINE_AA)

    cv2.imshow("MediaPipe Face Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

## Facial Landmark Detection

MediaPipe Face Mesh dapat mendeteksi hingga 478 titik landmark pada wajah manusia dengan presisi tinggi, memungkinkan analisis detail fitur wajah seperti mata, hidung, mulut, dan kontur wajah untuk berbagai aplikasi seperti deteksi emosi, tracking mata, dan augmented reality.

Berikut adalah list landmark beserta posisinya: https://storage.googleapis.com/mediapipe-assets/documentation/mediapipe_face_landmark_fullsize.png

### Deteksi wajah dan 468 titik landmark pada wajah

In [4]:
# Inisialisasi MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Konfigurasi Face Mesh
# static_image_mode=False untuk video streaming real-time
# max_num_faces=1 untuk mendeteksi maksimal 1 wajah
# refine_landmarks=True untuk mendeteksi iris mata juga (total 478 landmark)
# min_detection_confidence=0.5 confidence threshold untuk deteksi
# min_tracking_confidence=0.5 confidence threshold untuk tracking
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# Buka webcam (0 = default webcam)
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Gagal membuka webcam")

print("Tekan 'q' untuk keluar")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Gagal membaca frame dari webcam")
        break
    
    height, width = frame.shape[:2]
    
    # Convert BGR to RGB (MediaPipe membutuhkan RGB)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Process frame untuk deteksi face landmarks
    results = face_mesh.process(rgb)
    
    # Gambar landmarks jika wajah terdeteksi
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Gambar face mesh dengan koneksi antar landmark
            mp_drawing.draw_landmarks(
                image=frame,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_TESSELATION,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style()
            )
            
            # Gambar kontur wajah (bibir, mata, alis, wajah)
            mp_drawing.draw_landmarks(
                image=frame,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_CONTOURS,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_contours_style()
            )
            
            # Gambar iris mata (jika refine_landmarks=True)
            mp_drawing.draw_landmarks(
                image=frame,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_IRISES,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_iris_connections_style()
            )
    
    # Tampilkan FPS
    cv2.putText(frame, "Press 'q' to quit", (10, height - 20),
               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
    
    # Tampilkan frame
    cv2.imshow("MediaPipe Face Mesh - Webcam", frame)
    
    # Tekan 'q' untuk keluar
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()
face_mesh.close()

Tekan 'q' untuk keluar
Gagal membaca frame dari webcam


### Aplikasi sederhana: Deteksi Kedipan Mata (menggunakan rasio aspek mata / Eye Aspect Ratio)

Aplikasi deteksi kedipan mata menggunakan Eye Aspect Ratio (EAR) bekerja dengan menghitung rasio antara jarak vertikal dan horizontal mata dari landmark wajah. Berikut tahapan implementasinya:

**Tahapan Implementasi:**

1. **Ekstraksi Koordinat Mata**: Mengambil 6 titik landmark khusus untuk setiap mata (kiri dan kanan) dari 468 landmark wajah MediaPipe
2. **Perhitungan EAR**: Menghitung Eye Aspect Ratio menggunakan rumus: `EAR = (|p2-p6| + |p3-p5|) / (2 * |p1-p4|)` dimana p1-p6 adalah 6 titik landmark mata
3. **Threshold Detection**: Membandingkan nilai EAR dengan threshold (biasanya ~0.25) - jika EAR di bawah threshold berarti mata tertutup
4. **Frame Counting**: Menghitung berapa frame berturut-turut mata tertutup untuk menghindari false positive
5. **Blink Counter**: Increment counter kedipan ketika mata kembali terbuka setelah tertutup dalam durasi yang wajar

**Kegunaan**: Aplikasi ini berguna untuk sistem monitoring kantuk pengemudi, kontrol perangkat hands-free, atau analisis perhatian dalam pembelajaran online.

**Nilai EAR Umum**
- **Mata Terbuka**: ~0.3 - 0.4
- **Mata Tertutup**: ~0.2 - 0.3

**Pemilihan Landmark**
Kode ini menggunakan indeks landmark MediaPipe Face Mesh yang spesifik:
- **Mata Kanan**: [33, 159, 158, 133, 153, 145]
- **Mata Kiri**: [362, 380, 374, 263, 386, 385]

In [5]:
# Fungsi untuk menghitung Eye Aspect Ratio (EAR)
def calculate_ear(eye_landmarks):
    vertical_1 = np.linalg.norm(eye_landmarks[1] - eye_landmarks[5])
    vertical_2 = np.linalg.norm(eye_landmarks[2] - eye_landmarks[4])
    horizontal = np.linalg.norm(eye_landmarks[0] - eye_landmarks[3])
    
    # Hitung EAR
    ear = (vertical_1 + vertical_2) / (2.0 * horizontal)
    
    return ear

LEFT_EYE_INDEXES = [33, 160, 158, 133, 153, 144]
RIGHT_EYE_INDEXES = [362, 385, 387, 263, 373, 380]

# Threshold dan parameter
EAR_THRESHOLD = 0.2  # Threshold untuk mendeteksi mata tertutup
CONSECUTIVE_FRAMES = 2  # Jumlah frame berturut-turut untuk menghitung kedipan

# Counter
blink_counter = 0
total_blinks = 0

# Inisialisasi MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

In [6]:
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Gagal membuka webcam")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Gagal membaca frame dari webcam")
        break
    
    height, width = frame.shape[:2]
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb)
    
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Ekstrak koordinat landmark mata kiri
            left_eye = []
            for idx in LEFT_EYE_INDEXES:
                landmark = face_landmarks.landmark[idx]
                x = int(landmark.x * width)
                y = int(landmark.y * height)
                left_eye.append(np.array([x, y]))
                cv2.circle(frame, (x, y), 2, (0, 255, 0), -1)
            
            # Ekstrak koordinat landmark mata kanan
            right_eye = []
            for idx in RIGHT_EYE_INDEXES:
                landmark = face_landmarks.landmark[idx]
                x = int(landmark.x * width)
                y = int(landmark.y * height)
                right_eye.append(np.array([x, y]))
                cv2.circle(frame, (x, y), 2, (0, 255, 0), -1)
            
            # Hitung EAR untuk kedua mata
            left_ear = calculate_ear(left_eye)
            right_ear = calculate_ear(right_eye)
            
            # Rata-rata EAR
            avg_ear = (left_ear + right_ear) / 2.0
            
            if avg_ear < EAR_THRESHOLD:
                blink_counter += 1
            else:
                if blink_counter >= CONSECUTIVE_FRAMES:
                    total_blinks += 1
                blink_counter = 0
            
            cv2.putText(frame, f"EAR: {avg_ear:.2f}", (10, 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2, cv2.LINE_AA)
            
            status = "BERKEDIP" if avg_ear < EAR_THRESHOLD else "TERBUKA"
            color = (0, 0, 255) if avg_ear < EAR_THRESHOLD else (0, 255, 0)
            cv2.putText(frame, f"Status: {status}", (10, 60),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2, cv2.LINE_AA)
            
            cv2.putText(frame, f"Total Kedipan: {total_blinks}", (10, 90),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Gambar kontur mata\
            for eye in (left_eye, right_eye):
                for i in range(len(eye)):
                    pt1 = tuple(map(int, eye[i]))
                    pt2 = tuple(map(int, eye[(i + 1) % len(eye)]))
                    cv2.line(frame, pt1, pt2, (255, 0, 0), 1, cv2.LINE_AA)

    
    # Tampilkan frame
    cv2.imshow("Deteksi Kedipan Mata - Eye Aspect Ratio", frame)
    
    # Keyboard controls
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()
face_mesh.close()

print(f"Total kedipan terdeteksi: {total_blinks}")

Gagal membaca frame dari webcam
Total kedipan terdeteksi: 0


## Pose Landmark Detection

MediaPipe Pose Landmark Detection mendeteksi dan melacak 33 titik landmark pada tubuh manusia. memungkinkan komputer untuk "melihat" dan memahami postur serta gerakan tubuh manusia.

**33 Titik Landmark Tubuh:**
- **Wajah**: Hidung, mata kiri/kanan, telinga kiri/kanan (5 titik)
- **Tubuh Atas**: Bahu, siku, pergelangan tangan kiri/kanan (6 titik) 
- **Tubuh Tengah**: Pinggul kiri/kanan (2 titik)
- **Kaki**: Pinggul, lutut, pergelangan kaki, tumit, ujung kaki kiri/kanan (20 titik)

**Kegunaan Praktis:**
- **Analisis Olahraga**: Mengukur teknik gerakan atlet, mendeteksi postur yang salah
- **Fitness Apps**: Menghitung repetisi push-up, squat, atau latihan lainnya
- **Rehabilitasi Medis**: Monitoring progress pasien fisioterapi
- **Game & AR**: Kontrol karakter game menggunakan gerakan tubuh
- **Analisis Ergonomi**: Evaluasi postur kerja untuk mencegah cedera

### Deteksi 33 titik landmark pada tubuh manusia (skeleton)

In [7]:
# Inisialisasi MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# Inisialisasi VideoCapture
cap = cv2.VideoCapture(0)

while cap.isOpened():
    # Baca frame dari video
    ret, frame = cap.read()
    if not ret:
        print("Gagal mengambil frame. Keluar...")
        break

    # Ubah warna frame menjadi RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Deteksi pose landmark
    results = pose.process(frame_rgb)

    # Gambar landmark pada frame
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    # Tampilkan frame
    cv2.imshow('Pose Landmark Detection', frame)

    # Tekan 'q' untuk keluar
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Tutup VideoCapture dan jendela OpenCV setelah loop selesai
cap.release()
cv2.destroyAllWindows()


Gagal mengambil frame. Keluar...


### Aplikasi sederhana: Menghitung Sudut Siku Kanan

In [8]:
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    # Vector dari b ke a dan b ke c
    ba = a - b
    bc = c - b
    
    # Menghitung cosinus sudut menggunakan dot product
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    
    # Menghindari error numerical
    cosine_angle = np.clip(cosine_angle, -1.0, 1.0)
    
    # Konversi ke derajat
    angle = np.arccos(cosine_angle)
    angle = np.degrees(angle)
    return angle

In [9]:
BAHU_KANAN, SIKU_KANAN, TELAPAK_KANAN = 12, 14, 16

mp_pose = mp.solutions.pose
mp_draw = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, model_complexity=1,
                    enable_segmentation=False, min_detection_confidence=0.5,
                    min_tracking_confidence=0.5)

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Gagal membuka webcam")

while True:
    ok, frame = cap.read()
    if not ok:
        print("Gagal membaca frame")
        break

    height, width = frame.shape[:2]
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    res = pose.process(rgb)

    if res.pose_landmarks:
        landmark = res.pose_landmarks.landmark

        if landmark[BAHU_KANAN].visibility > 0.5 and landmark[SIKU_KANAN].visibility > 0.5 and landmark[TELAPAK_KANAN].visibility > 0.5:
            ls = [landmark[BAHU_KANAN].x * width, landmark[BAHU_KANAN].y * height]
            rs = [landmark[SIKU_KANAN].x * width, landmark[SIKU_KANAN].y * height]
            re = [landmark[TELAPAK_KANAN].x * width, landmark[TELAPAK_KANAN].y * height]

            angle_r = calculate_angle(ls, rs, re)

            cv2.circle(frame, tuple(map(int, ls)), 6, (0, 255, 0), -1, cv2.LINE_AA)
            cv2.circle(frame, tuple(map(int, rs)), 6, (0, 0, 255), -1, cv2.LINE_AA)
            cv2.circle(frame, tuple(map(int, re)), 6, (0, 255, 0), -1, cv2.LINE_AA)
            cv2.line(frame, tuple(map(int, ls)), tuple(map(int, rs)), (255, 255, 255), 3, cv2.LINE_AA)
            cv2.line(frame, tuple(map(int, rs)), tuple(map(int, re)), (255, 255, 255), 3, cv2.LINE_AA)

            cv2.putText(frame, f"Siku Kanan: {angle_r:.1f}°", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow("Deteksi Sudut Siku Kanan", frame)

    if (cv2.waitKey(1) & 0xFF) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
pose.close()

Gagal membaca frame


### 🔎 Eksplorasi

- Cobalah mendeteksi bahu kiri
- Cobalah mendeteksi pose tubuh (berdiri, jongkok, tidur)

## **Ringkasan Materi: Video Processing Week 2 - Analisis Manusia dengan MediaPipe**

### **🎯 Tujuan Pembelajaran**
Mempelajari analisis manusia dalam video menggunakan teknologi AI melalui MediaPipe untuk deteksi dan tracking fitur wajah serta pose tubuh manusia secara real-time dengan akurasi tinggi.

### **📚 Materi yang Dipelajari**

#### **1. MediaPipe Framework**
- Framework open-source dari Google untuk pemrosesan media real-time
- Menyediakan model deteksi wajah yang ringan, akurat, dan cepat
- Dapat digunakan untuk gambar statis maupun video streaming real-time

#### **2. Facial Landmark Detection**
- **Deteksi Wajah**: Menggunakan MediaPipe Face Detection dengan bounding box
- **468 Titik Landmark**: MediaPipe Face Mesh mendeteksi hingga 478 titik landmark pada wajah
- **Aplikasi Praktis**: Deteksi kedipan mata menggunakan Eye Aspect Ratio (EAR)
    - Rumus EAR: `(|p2-p6| + |p3-p5|) / (2 * |p1-p4|)`
    - Threshold EAR: ~0.2-0.3 (mata tertutup), ~0.3-0.4 (mata terbuka)

#### **3. Pose Landmark Detection**
- **33 Titik Landmark**: Deteksi dan pelacakan titik-titik kunci pada tubuh manusia
- **Distribusi Landmark**:
    - Wajah: 5 titik (hidung, mata, telinga)
    - Tubuh atas: 6 titik (bahu, siku, pergelangan tangan)
    - Tubuh tengah: 2 titik (pinggul)
    - Kaki: 20 titik (pinggul hingga ujung kaki)
- **Aplikasi**: Analisis sudut siku untuk deteksi gerakan

### **💡 Aplikasi Praktis**
- **Monitoring Kesehatan**: Deteksi kantuk pengemudi melalui kedipan mata
- **Fitness & Olahraga**: Analisis teknik gerakan dan penghitungan repetisi
- **Rehabilitasi Medis**: Monitoring progress pasien fisioterapi
- **Augmented Reality**: Kontrol perangkat menggunakan gerakan tubuh
- **Analisis Ergonomi**: Evaluasi postur kerja untuk pencegahan cedera

### **🔧 Library yang Digunakan**
- **OpenCV**: Operasi computer vision dasar
- **MediaPipe**: Model AI untuk deteksi wajah dan pose
- **NumPy**: Komputasi numerik dan manipulasi array
- **Matplotlib**: Visualisasi hasil

### **📈 Hasil Pembelajaran**
Mampu membangun aplikasi computer vision yang dapat:
1. Mendeteksi wajah dan fitur wajah secara real-time
2. Menganalisis kedipan mata dengan akurasi tinggi
3. Mendeteksi pose tubuh dan mengukur sudut sendi
4. Mengimplementasikan solusi praktis untuk analisis gerakan manusia